In [67]:
import xml.etree.ElementTree as ET

class PDBTM:
    def __init__(self, id = str, chains = []): 
        self.id = id
        self.chains = chains
    
class Chain:
    def __init__(self, id = str, num_tm = int, typ = str, seq = str, regions = []): 
        self.id = id
        self.num_tm = num_tm
        self.typ = typ
        self.regions = regions
    
class Region:
    def __init__(self, seq_beg = int, pdb_beg = int, seq_end = int, pdb_end = int, typ = str):
        self.seq_beg = seq_beg
        self.pdb_beg = pdb_beg
        self.seq_end = seq_end
        self.pdb_end = pdb_end
        self.typ = typ

In [128]:
import os
from shutil import copyfile

def filter_for_tmps(path):
    counter = 0
    for root, dirs, files in os.walk(path):
        for name in files:
            if name.endswith(".xml"): 
                path = os.path.join(root,name)
                pdbtm_xml = ET.parse(path) 
                pdbtm_root = pdbtm_xml.getroot()
                is_tmp = pdbtm_root.attrib.get('TMP')
                if is_tmp == 'yes':
                    print("Copy", pdbtm_root.attrib.get("ID"))
                    copyfile(path, os.path.join("pdbtm_xmls",name))
                    counter = counter + 1
                        
    return counter
                
#number_of_tms = filter_for_tmps("pdbtm/database")    
print(number_of_tms)

Copy 1y2l
Copy 4y25
Copy 4y28
Copy 2y2x
Copy 2v61
Copy 2v60
Copy 3pcv
Copy 1pcr
Copy 3pcq
Copy 3vzw
Copy 3vzt
Copy 3vzu
Copy 1vz1
Copy 2zqp
Copy 1uun
Copy 3uu8
Copy 4uu1
Copy 4uu0
Copy 2uui
Copy 2uuh
Copy 3uu6
Copy 3uu4
Copy 3uub
Copy 4uuj
Copy 3uu5
Copy 3uu2
Copy 3uu3
Copy 3slo
Copy 3slj
Copy 3slt
Copy 4u9n
Copy 4u9l
Copy 5u9w
Copy 4u96
Copy 4u95
Copy 4pdl
Copy 4pdm
Copy 3pds
Copy 4pd9
Copy 4pd8
Copy 4pdr
Copy 4pd5
Copy 4pd4
Copy 4pd6
Copy 4pda
Copy 4pd7
Copy 4pdv
Copy 4v1g
Copy 4v1f
Copy 4v1h
Copy 1y5i
Copy 1y5l
Copy 1y5n
Copy 2y5y
Copy 1y5d
Copy 1skh
Copy 1zv0
Copy 5u76
Copy 5u70
Copy 1u7g
Copy 1u77
Copy 1u7c
Copy 4z3p
Copy 4z34
Copy 4z35
Copy 4z36
Copy 4z3n
Copy 2v8n
Copy 3v8g
Copy 3v8f
Copy 3v8x
Copy 3v89
Copy 2vt4
Copy 2ypt
Copy 2zxw
Copy 2zxe
Copy 5u09
Copy 4pj0
Copy 3pjz
Copy 3pjs
Copy 4zjl
Copy 4zjo
Copy 4zj8
Copy 2zjs
Copy 3zjz
Copy 4zjq
Copy 4zjc
Copy 5unf
Copy 5ung
Copy 5uni
Copy 5unh
Copy 2yev
Copy 4yeu
Copy 1yew
Copy 5vai
Copy 5va1
Copy 5va2
Copy 5va3
Copy 4px7
Copy 3pxo


Copy 5kkz
Copy 5kk2
Copy 4kkl
Copy 4kk9
Copy 4kk8
Copy 4bv0
Copy 4bvn
Copy 3bvd
Copy 4dop
Copy 5doq
Copy 5do7
Copy 1dop
Copy 4doj
Copy 2hvj
Copy 2hvk
Copy 5hvd
Copy 5hv9
Copy 5hvx
Copy 3nog
Copy 3noc
Copy 2nop
Copy 2now
Copy 2gr8
Copy 2grx
Copy 2gr7
Copy 4grv
Copy 2aki
Copy 2akh
Copy 5kyh
Copy 2kyv
Copy 4ky0
Copy 1kyk
Copy 4kyt
Copy 1kyo
Copy 1kyl
Copy 2kyh
Copy 2k58
Copy 2k59
Copy 4k5y
Copy 4d1b
Copy 4d1c
Copy 4d1a
Copy 4d1d
Copy 4al0
Copy 4al1
Copy 1gue
Copy 1gu8
Copy 2guf
Copy 1g90
Copy 4nh2
Copy 4hqj
Copy 2hqg
Copy 2hqf
Copy 2hqd
Copy 2hqc
Copy 5d6l
Copy 5d6i
Copy 5d6k
Copy 4d65
Copy 4d6t
Copy 4d6u
Copy 4d64
Copy 3dzm
Copy 1dze
Copy 2bcc
Copy 3bcc
Copy 1bct
Copy 1bcc
Copy 1k24
Copy 5k2a
Copy 5k2c
Copy 5k2b
Copy 5k2d
Copy 2k21
Copy 1mgy
Copy 5mg3
Copy 2mgy
Copy 5dsg
Copy 1ds8
Copy 3dsy
Copy 2mnh
Copy 4mnd
Copy 2mn7
Copy 2mn6
Copy 2kwx
Copy 3aeh
Copy 3ae9
Copy 3ae8
Copy 5aex
Copy 5aez
Copy 3ae5
Copy 3aec
Copy 3aeb
Copy 3ae4
Copy 3ae6
Copy 3aea
Copy 3ae7
Copy 3aee
Copy 3ae3
Copy 3ae2


Copy 2x6b
Copy 2x6c
Copy 2x6a
Copy 2w2e
Copy 5tvn
Copy 4ro2
Copy 3w5a
Copy 3w5c
Copy 3w5b
Copy 1w5c
Copy 3w5d
Copy 2w5j
Copy 4x1h
Copy 2wp9
Copy 1wp1
Copy 1wpg
Copy 2wpd
Copy 2qi9
Copy 4qin
Copy 4qim
Copy 4qi1
Copy 4qiq
Copy 4qid
Copy 4xtl
Copy 4xtn
Copy 4xto
Copy 4xtc
Copy 2xtv
Copy 2xtu
Copy 4x8a
Copy 4x88
Copy 4x89
Copy 5t36
Copy 5t37
Copy 5t3r
Copy 3rfz
Copy 4rfs
Copy 3rfm
Copy 3rfu
Copy 3qnq
Copy 4qnz
Copy 4qnd
Copy 4qnc
Copy 4ww3
Copy 1txh
Copy 3tx3
Copy 3txt
Copy 4rai
Copy 5t4p
Copy 5t4q
Copy 5t4d
Copy 3t4d
Copy 3t45
Copy 3t4z
Copy 5t4y
Copy 5t4o
Copy 5uj9
Copy 1ujw
Copy 5uja
Copy 5vew
Copy 5vex
Copy 4p02
Copy 4p00
Copy 3p03
Copy 3p0g
Copy 4yay
Copy 4ziw
Copy 4ziv
Copy 4zit
Copy 2ziy
Copy 3stl
Copy 3stz
Copy 4umv
Copy 4umw
Copy 1umx
Copy 3um7
Copy 3s8f
Copy 3s8g
Copy 1s8l
Copy 1s8j
Copy 2yfy
Copy 1yf6
Copy 2p7t
Copy 1p7b
Copy 4p79
Copy 5vb2
Copy 5vbl
Copy 5vb8
Copy 1yo9
Copy 2yoe
Copy 4pr7
Copy 1prn
Copy 3prn
Copy 6prc
Copy 2prc
Copy 7prn
Copy 5prn
Copy 5prc
Copy 7prc
Copy 2prn


Copy 2m3e
Copy 2m3g
Copy 1m3x
Copy 1kfy
Copy 4kfm
Copy 1kf6
Copy 5gmy
Copy 2gmr
Copy 2atk
Copy 2at9
Copy 4atv
Copy 1at9
Copy 1aty
Copy 4a82
Copy 5a8e
Copy 2hi7
Copy 2hit
Copy 5hi9
Copy 2npj
Copy 2npk
Copy 4npp
Copy 2npe
Copy 2npd
Copy 2npg
Copy 2npc
Copy 5kai
Copy 2ka1
Copy 2ka2
Copy 1kad
Copy 5kaf
Copy 2mxb
Copy 4m48
Copy 3m4d
Copy 3m4e
Copy 3det
Copy 5b0w
Copy 2b0x
Copy 1nw0
Copy 2nwl
Copy 2nwx
Copy 2nww
Copy 2hn2
Copy 2asg
Copy 3asn
Copy 3aso
Copy 3gjc
Copy 3gjd
Copy 5gjv
Copy 5gjw
Copy 2hg5
Copy 2hg3
Copy 4hg6
Copy 2hg9
Copy 2n2m
Copy 2n2l
Copy 3n23
Copy 2n2a
Copy 4gcq
Copy 4gc0
Copy 4gcp
Copy 4gcs
Copy 4azl
Copy 5azc
Copy 5azb
Copy 5azd
Copy 5a6u
Copy 5a6g
Copy 5a63
Copy 5a6e
Copy 2a65
Copy 4mqx
Copy 4mqt
Copy 4mqs
Copy 4khz
Copy 5khs
Copy 5khn
Copy 3b9z
Copy 3b9y
Copy 3b9r
Copy 1b9u
Copy 3b9b
Copy 3b9w
Copy 4bum
Copy 4buo
Copy 5but
Copy 5dl5
Copy 5dl6
Copy 5dl7
Copy 5dl8
Copy 3dl8
Copy 1a11
Copy 5a1s
Copy 3gd8
Copy 4gd3
Copy 2n5s
Copy 3n5k
Copy 3nym
Copy 3ny8
Copy 3ny9
Copy 4nyk


In [89]:
def parse_pdbtm_xmls(paths):
    '''
    Parses PDBTM XML files to a list od PDBTM Objects.
    A PDBTM Object contains the PDB ID as well as a list of Chain objects. 
    A Chain Object contains the chain ID, the tm number, the type, the sequence and a list of Region objects.
    A Region object contains the begin and end indices of the sequence and the pdb file, as well as the type.
    '''
    ns = {'pdbtm': 'http://pdbtm.enzim.hu'}
    pdbtms = []
    for pdbtm_xml in pdbtm_xmls:
        pdbtm_xml = ET.parse(pdbtm_xml) 
        pdbtm_root = pdbtm_xml.getroot()
        pdbtm_id = pdbtm_root.attrib.get('ID')
        
        if pdbttm_root.attrib.get('TMP') == 'yes':

            chains = []
            for chain_xml in pdbtm_root.findall('pdbtm:CHAIN', ns):
                chain_id = chain_xml.attrib.get('CHAINID')
                num_tm = chain_xml.attrib.get('NUM_TM')
                typ = chain_xml.attrib.get('TYPE')
                seq = chain_xml.find('pdbtm:SEQ', ns)

                regions = []
                for region_xml in chain_xml.findall('pdbtm:REGION', ns):
                    seq_beg = region_xml.attrib.get('seq_beg')
                    pdb_beg = region_xml.attrib.get('pdb_beg')
                    seq_end = region_xml.attrib.get('seq_end')
                    pdb_end = region_xml.attrib.get('pdb_end')
                    typ_region = region_xml.attrib.get('type')
                    region = Region(seq_beg, pdb_beg, seq_end, pdb_end, typ)
                    regions.append(region)
            chain = Chain(chain_id, num_tm, typ_region, seq, regions)
            chains.append(chain) 

            pdbtm = PDBTM(pdbtm_id, chains)
            pdbtms.append(pdbtm)
        
        else:
            print(pdbtm_id, "is no TMP and was ignored.")
    
    return pdbtms

In [90]:
pdbtm_xmls = ['test/pdbtmalpha']
pdbtms = parse_pdbtm_xmls(pdbtm_xmls)

for pdbtm in pdbtms:
    print(pdbtm.id)

ParseError: XML or text declaration not at start of entity: line 32944, column 0 (<string>)